## Super-Resolution Image Generation Class

The purpose of this notebook is to develop a class which can be consistently use for image handling when working with Super-Resolution (SR) models (mainly on MRI or other 3D medical images). Since the problem that Super-Resolution models are aiming to solve is fairly consistent, there should be significant overlap in how testing and training images are created/handled.


### DISCLAIMER:
This class is meant to make playing around with SR models easier on *personal* computing hardware. This class is not designed with large computing resources in mind and will likely be less efficient than other methods. There are three particular issues related to playing around with SR that this class is meant to address:
1. **Overhead involved with augmentation of training and testing images**. When bug-testing and playing around with hyperparameters, the overhead of augmentating images, taking patches, etc. add time that isn't necessary. By generating these images/sub-images ahead of time the local memeory is leveraged to skip these steps each time you re-run after changing something. Once everything is working, it's easy to keep generating new augmented images as the model is trained.

2. **RAM limitations on personal devices**. Medical/3D images can be painfuly large. If you are training a model on sub-samples/patches of larger files, you either have to only sample from a subset of images for a batch, then load another subset for the next batch (hindering how randomly sampled the contents of each batch are), or inefficiently load images randomly only to take 1-2 sub-samples/patches from them. By saving these sub-samples/patches as seperate files ahead of time, random sampling is easier and less demanding of resources.

3. **Data organization**. It is very easy to get turned around when generating the low-resolution and high-resolution image pairs for training and testing. This class enforces a consistent organizational method, which allows it to take inventory and find matching images from previously generated images.


### Functionality:
To prevent this class from becoming a bloated mess of functionality and niche features, the ideal capabilities of this class will be cataloged here. The goals for this tool are:
1. Given an input directory, list all files that match a particular `prefix` and `suffix`
    - Detect if the directory(s) given were generated by this class previously
2. Load common image types:
    - [X] `.png`
    - [X] `.nii` and `.nii.gz`
    - [] `.dcm`
3. Support several different methods of downsampling and upsampling:
    - [X] bilinear interpolation
    - [] trilinear interpolation
    - [] "nearest" (see pytorch torchvision)
4. Create randomly shuffled/altered images at different resolutions (gaussian blur, affine transformation, etc.)
    - Make the aspect of saving these images optional so it can be used as a `generator` if possible
5. Display sample images from the generated images for assurance when prompted
6. Save files in multiple formats, when relevant:
    - [X] 2D full image (potentially transformed)
    - [X] 2D patch
    - [X] 3D full image (potentially transformed)
    - [X] 3D "patch", or a sub-volume for the 
7. Save any image generated in a specified format
8. Have locations of matching low and high resolution images
9. Generate file containing the values used in the template for future revisting.

## Import necessary libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from tqdm import tqdm
from pathlib import Path
import shutil
import numpy as np
from PIL import Image
import random
import cv2
import math as math
import nibabel as nib
import pydicom
from matplotlib import pyplot
from skimage.transform import rotate, AffineTransform, warp, rescale, resize

## Class definition

In [ ]:
# The super resolution class:

class sr_gen():
    def __init__(self, inp_dir, HR_out_dir, LR_out_dir, prefix='', suffix=''):
        self.inp_dir = inp_dir
        self.HR_out_dir = HR_out_dir
        self.HR_files = None
        self.LR_out_dir = LR_out_dir
        self.LR_files = None
        self.inp_files = self._get_inp_(prefix, suffix)
        self.template = self.get_template()

    def _get_inp_(self, prefix='', suffix=''):
        # Get the original files that will be used to generate everything
        # Based on the prefix, for now this could be nifti files or png's.
        # Will have to write a "try" statement in order to check for file type
        files = []
        for fil in os.listdir(self.inp_dir):
            if fil.startswith(prefix) & fil.endswith(suffix):
                files.append(fil)
        
        if not files:
            raise FileNotFoundError('No applicable files found in input directory')

        return files

    def _get_LR_out_(self):
        # get list of files in output directory and determine matching files
        return os.listdir(self.LR_out_dir)

    def _get_HR_out_(self):
        return os.listdir(self.HR_out_dir)


    def _view_sample_(self):
        # Function which loads and displays random example image for sanity check
        pyplot.figure()
        f, axes = pyplot.subplots(2,2)
        
        indx = random.randint(0,len(self.inp_files)-1)
        indy = random.randint(0,len(self.HR_files)-1)

        axes[0,0].imshow(np.array(Image.open(self.inp_dir+self.inp_files[indx])))
        axes[1,0].imshow(np.array(Image.open(self.HR_files[indy])))
        axes[1,1].imshow(np.array(Image.open(self.LR_files[indy])))


    def get_template(self):
        # Returns dictonary of all option settings for this class
        try:
            return self.template
        except:
            return {'out_type':'png', # png, nii (?), DICOM (?)
                    'unit':'intensity', #Whether you want RBG or Intensity/DICOM units
                    'resolution':2,
                    'translation_x':10,
                    'translation_y':10,
                    'rotation':0,
                    'scale':2,
                    'patch':False,
                    'step': 10,
                    'keep_blank':False,
                    }

    def save_template(self, temp):
        # apply the provided template for randomization to self for access by other functions
        self.template = temp


    def run(self, clear=False):
        # Run the analysis specified in the template dictionary. If clear is true then the 
        # files in the output directories will be deleted before creating the new images.

        if clear:
            print('Clearing existing output directories')
            shutil.rmtree(self.HR_out_dir, ignore_errors=True)
            shutil.rmtree(self.LR_out_dir, ignore_errors=True)

        # Make the directories where the new files will be saved
        os.makedirs(self.HR_out_dir, exist_ok=True)
        os.makedirs(self.LR_out_dir, exist_ok=True)

        HR_out_files = []
        LR_out_files = []
        
        s = 1/self.template['resolution']
        for im in self.inp_files:
            im_h = np.array(Image.open(self.inp_dir + im))

            # check the dimensions of the image
            #print(f'Shape of High Resolution Image:{im_h.shape}')

            im_h = self.rgb2ycrbcr(im_h)
            im_h = im_h[:,:,0] #Just deal with intensity values at the moment because 
                                # having multiple channels throws off cv2 when saving, 
                                # since it also does BGR instead of RGB and will save a blue image

            # TODO: compare cv2.resize with skimage.rescale or pytorch rescale for this
            im_l = cv2.resize(im_h, (0,0), fx = s, fy =s, interpolation=cv2.INTER_CUBIC)
            im_l = cv2.resize(im_l, (0,0), fx = self.template['resolution'],
            fy=self.template['resolution'], interpolation=cv2.INTER_CUBIC)
            # TODO: The above resizing results in values outside of the range [0, 255] due to 
            # the INTER_CUBIC method. For now I'm just clipping the values, but a more nuanced
            # answer should be found
            im_l = np.clip(im_l, 0, 255)

            if self.template['patch']:
                im, im_h, im_l = self.img_transform(im, im_h, im_l)
                _ = self.img2patches(im, im_h, im_l, keep_blank = self.template['keep_blank'],save=True)
                HR_out_files = HR_out_files + _

            else:
                _ = self.img_transform(im, im_h, im_l, save=True)
                HR_out_files.append(_)
        
        #Do LR first because otherwise HR_out_files is changed
        LR_out_files = [self.LR_out_dir + s for s in HR_out_files]
        HR_out_files = [self.HR_out_dir + s for s in HR_out_files]

        self.HR_files = HR_out_files
        self.LR_files = LR_out_files


    def img_transform(self,im, im_h, im_l, save=False):
        # Transform the original files using a variety of methods

        opp = '' #string for storing the operations performed on the images

        # If shifting in the x or y direction was selected
        if self.template['translation_x'] > 0 | self.template['translation_y'] > 0:
            _a = np.random.randint(0,self.template['translation_x'])
            _b = np.random.randint(0,self.template['translation_y'])
            transform = AffineTransform(translation=(_a, _b))
            im_h = warp(im_h, transform,mode='reflect')
            im_l = warp(im_l, transform,mode='reflect')
            opp += f'_x{_a}_y{_b}'

        if self.template['scale'] > 1:
            _a = np.random.randint(1,self.template['scale']+1)
            #transform = AffineTransform(scale=_a)
            im_h = rescale(im_h, scale = _a, mode='reflect')
            im_l = rescale(im_l, scale = _a, mode='reflect')
            opp+= f'_scale{_a}'

        # If rotation was selected
        if self.template['rotation'] > 0:
            _a = np.random.randint(-self.template['rotation'],self.template['rotation'])
            im_h = rotate(im_h, _a, mode="reflect")
            im_l = rotate(im_l, _a, mode="reflect")
            opp+= f'_rot{_a}'

        opp = im.split('.')[0] + opp

        if save:
            print(f'Saving image: {opp}')
            cv2.imwrite(f'{self.HR_out_dir}/{opp}.png', im_h)
            cv2.imwrite(f'{self.LR_out_dir}/{opp}.png', im_l)
            return opp
        else:
            return opp, im_h, im_l

    def img2patches(self, im, im_h, im_l=False, keep_blank=False, save=False):
        # Take a given image and generate patches and returns a stack of images
        # im : str, name of the image being cut into patches
        # im_h : ndarray, numpy array of the high-resolution image
        # im_l : ndarray, numpy array of the low-resolution image. If this is false, 
        # then you only want to make patches from one image (in this case im_h)

        patch_size = self.template['patch']
        step = self.template['step']


        # Get the height and width of the provided images
        h_h, w_h = im_h.shape
        h_l, w_l = im_l.shape

        # Create a numpy stack following Pytorch protocols
        HR_stack = np.zeros((len(range(0,w_h,step))*len(range(0,h_h,step)),patch_size,patch_size))
        if isinstance(im_l, np.ndarray):
            LR_stack = np.zeros((len(range(0,w_h,step))*len(range(0,h_h,step)),patch_size,patch_size))

        im_name = im.split('.')[0]

        cnt = 0
        blank = 0

        for i in range(0,w_h,step):
            for j in range(0,h_h,step):
                if i+patch_size < w_h and j+patch_size < h_h:

                    sample_h = im_h[j:j+patch_size, i: i+patch_size]
                    if isinstance(im_l, np.ndarray):
                        sample_l = im_l[j:j+patch_size, i:i+patch_size]

                    # if you've chosen to keep blank patches or if the patch is not blank add 
                    # it to the stack
                    if keep_blank or (sample_h.max() > 0 or sample_l.max() > 0):
                        HR_stack[cnt, :,:] = sample_h
                        if isinstance(im_l, np.ndarray):
                            LR_stack[cnt,:,:] = sample_l
                        cnt += 1
                    else:
                        blank += 1



        # Return a list of image names and numpy array with the first cnt layers
        HR_fnames = []
        for i in range(cnt):
            HR_fnames.append(f'{im_name}_{i}.png')

        if save: #Whether to save a patch if it is blank/intensity value of 0
            for i in range(cnt):
                cv2.imwrite(f'{self.HR_out_dir}/{im_name}_{i}.png', HR_stack[i,:,:])
                if isinstance(im_l, np.ndarray):
                    cv2.imwrite(f'{self.LR_out_dir}/{im_name}_{i}.png', LR_stack[i,:,:])
            return HR_fnames
        
        else:
            if isinstance(im_l, np.ndarray):
                return HR_fnames, HR_stack[:cnt,:,:], LR_stack[:cnt,:,:]
            else:
                return HR_fnames, HR_stack[:cnt,:,:]
        


    def rgb2ycrbcr(self, img_rgb):
        # Takes an RBG image and returns it as a YCRBCR image (if you just want to focus
        #  on luminance values of an image)

        img_rgb = img_rgb.astype(np.float32)

        img_ycrcb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YCR_CB)
        img_ycbcr = img_ycrcb[:,:,(0,2,1)].astype(np.float32)
        img_ycbcr[:,:,0] = (img_ycbcr[:,:,0]*(235-16)+16)/255.0
        img_ycbcr[:,:,1:] = (img_ycbcr[:,:,1:]*(240-16)+16)/255.0

        return img_ycbcr


    def load_image_pair(self, im_id):
        # A method which loads the provided file and returns a numpy array
        # this is used because it will remember in the template dictionary how
        # the images were saved (either as RBG or intensity values or 3D array). 
        # This will help minimize headaches caused by different image types.

        # im_id can either be the index value or the name of the file
        if isinstance(im_id, int):
            HR_file = self.HR_files[im_id]
            LR_file = self.LR_files[im_id]
        elif isinstance(im_id, str):
             _ = self.HR_files.index(im_id)
             HR_file = self.HR_files[_]
             LR_file = self.LR_files[_]
        else:
            TypeError("Invalid image identifier, please input a string to integer")

        # Check what data type to load from the template
        if self.template['out_type'] == 'png':
            im_h = np.array(Image.open(HR_file))
            im_l = np.array(Image.open(LR_file))


        return im_h, im_l

    def match_altered(self, update=True, paths=False):
        # Get the files that have been generated in the output directory
        # If update is false, then just return a list of matched names, if true then
        # change the class variable values accordingly.
        hr_files = self._get_HR_out_()
        lr_files = self._get_LR_out_()

        # Get a set of all the files with agreement before the metadata
        if len(hr_files) > len(lr_files):
            matches = list(set(hr_files)-(set(hr_files)-set(lr_files)))
        else:
            matches = list(set(lr_files)-(set(lr_files)-set(hr_files)))

        if update:
            # If you want to save these matched files as class variables
            self.HR_files = [self.HR_out_dir + _ for _ in matches]
            self.LR_files = [self.LR_out_dir + _ for _ in matches]
            print('HR and LR file locations updated')
        
        if paths:
            return self.HR_files, self.LR_files

    def change_out(self, HR_out_dir, LR_out_dir):
        # Change the output locations so you can save into a new file
        self.HR_out_dir = HR_out_dir
        self.HR_files = None
        self.LR_out_dir = LR_out_dir
        self.LR_files = None

In [ ]:
qq = sr_gen('./data/raw/nii_sub_HR/','./data/raw/HR_output/','./data/raw/LR_output/')

In [ ]:
#qq.match_altered(True)
#pyplot.imshow(qq.load_image_pair(4)[1])

In [ ]:
qq.run(clear=True)

In [ ]:
qq.match_altered(update=True)
qq._view_sample_()

## Experimentation with Nifti loading and saving

In [ ]:
import nibabel as nib
# https://nipy.org/nibabel/gettingstarted.html

#nibabel load gets the shape and data type without needing to load anything. To do that you need to call get_fdata()
img = nib.load('../data/nifti/MPRAGE_Structural_Scan.nii')

In [ ]:
#Get information from the header of img
img.header.get_xyzt_units()

In [ ]:
from matplotlib import pyplot
data = img.get_fdata()

# dimensions are [coronal,axial,sagittal]
#pyplot.imshow(warp(data[:,90,:],AffineTransform(rotation=6)))
pyplot.imshow(rotate(data[:,:,70], 10))
pyplot.show()

In [ ]:
# Apply transformations to the data
_ = rescale(data, scale=(2,2,2), mode='symmetric')
pyplot.imshow(_[:,:,180])
pyplot.show()

In [ ]:
# Translation

#okay, for some reason, scikitimage does not work with 3D as intended, but will accept it for some reason....
trans = np.eye(3)
trans[:2,2]=40 #move 40 in the coronal and axial directions (i think?)
transform = AffineTransform(matrix = trans)
_ = warp(data, transform, mode = 'reflect')
pyplot.imshow(_[:,:,90])
pyplot.show()

In [ ]:
#Going to need to normalize the nifti data into some consistent range...
_.max()

In [ ]:
data.shape

In [ ]:
#rotation
samp_dat = np.array([[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]]])
#im_h = rotate(im_h, _a, mode="reflect")
rotate(samp_dat,angle=0)

In [ ]:
# Try to do rotation in another way
from math import sin, cos, radians
trans = np.eye(3)
theta = radians(20)
trans[0,0]=cos(theta)
trans[0,1]=-sin(theta)
trans[1,0]=sin(theta)
trans[1,1]=cos(theta)

#For just rotation along the sagittal plane with no scaling:
# [cos(), -sin(), 0]
# [sin(), cos(), 0]
# [0, 0, 1]

transform = AffineTransform(matrix = trans)
_ = warp(data, transform, mode = 'reflect')
pyplot.imshow(_[:,:,90])
pyplot.show()

In [ ]:
# Save selection of nifty file as its own nifti file
qq = nib.Nifti1Image(data[:,:,90], np.eye(4))
nib.save(qq, './practice_nifti.nii.gz')

In [ ]:
img = nib.load('./practice_nifti.nii.gz') #test loading of nii.gz files
data = img.get_fdata()
pyplot.imshow(data)
pyplot.show()

## Experimentation with DICOM loading and saving

In [ ]:
import pydicom as dicom

In [ ]:
# Load single dicom image
ds = dicom.dcmread('../data/dicom/A0009')
(2,) + ds.pixel_array.shape


In [ ]:
# Load stack of .dcm images
image_path = '../data/dicom/'

im_list = sorted(os.listdir(image_path)) #SORTING IS VERY IMPORTANT FOR 3D IMAGES IN DICOM

In [ ]:
stk = np.zeros((len(im_list),)+dicom.dcmread(image_path+im_list[0]).pixel_array.shape) # weird thing I have to do to make (h,w) into (s, h, w) because you can add to tuples 
for idx, f in enumerate(im_list):
    #print(f)
    stk[idx,:,:] = dicom.dcmread(image_path + f).pixel_array

In [ ]:
pyplot.imshow(stk[41,:,:])

# Class Definition V2.0
With the previous version of the class done, and some lesson learned from actually implementing things

In [56]:
from re import template
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from tqdm import tqdm
from pathlib import Path
import shutil
import numpy as np
from PIL import Image
import random
import cv2
import math as math
import nibabel as nib
import pydicom
from matplotlib import pyplot
from skimage.transform import rotate, AffineTransform, warp, rescale, resize

class SrGen:
    def __init__(self, inp_dir, HR_out_dir, LR_out_dir, prefix='', suffix=''):
        self.inp_dir = inp_dir
        self.HR_out_dir = HR_out_dir
        self.HR_files = None
        self.LR_out_dir = LR_out_dir
        self.LR_files = None
        self.inp_files, self.inp_paths = self._get_inp_(prefix, suffix)
        self.template = self.get_template()

### ORANIZATIONAL METHODS
    def _get_inp_(self, prefix='', suffix=''):

        files = []
        paths = []
        # If they have provided a list of directories (in the case of DICOM or scattered data)
        if isinstance(self.inp_dir, list):
            for inp_dir in self.inp_dir:
                for fil in os.listdir(inp_dir):
                    if fil.startswith(prefix) and fil.endswith(suffix):
                        paths.append(inp_dir + fil)
                        files.append(fil)

                    if not files:
                        raise FileNotFoundError('No applicable files found in input directory')
        else:
            for fil in os.listdir(self.inp_dir):
                if fil.startswith(prefix) and fil.endswith(suffix):
                    paths.append(self.inp_dir + fil)
                    files.append(fil)

                if not files:
                    raise FileNotFoundError('No applicable files found in input directory')

        return files, paths
    
    def _get_LR_out_(self):
        # get list of files in output directory and determine matching files
        return os.listdir(self.LR_out_dir)

    def _get_HR_out_(self):
        return os.listdir(self.HR_out_dir)

    def get_template(self):
        try:
            return self.template
        except:
            return {'out_type':'png',
                'unit':'intensity', #Currently only matters for png
                'resolution':None,
                'same_size': True, # Whether to have the LR image be the same size as the HR image
                                   # (i.e. whether to scale down then up or just down)
                'translation':None, # Have both single value or multiple
                'rotation': None, # Around each axis
                'scale': False, # What magnitude to zoom in for added jitter
                'patch': False, # Have this accept 3 dimensional input [x,y,z], [x,y], or single
                'step': 10, # Also have this accept 3 dimensional input
                'keep_blank': False,
            }
    
    def set_template(self, temp):
        self.template = temp

    def load_img_pair(self, im_id):
        # 
        if isinstance(im_id, int):
            im_h = self.load_image(self.HR_files[im_id])
            im_l = self.load_image(self.LR_files[im_id])
        elif isinstance(im_id, str):
            _ = self.HR_files.index(im_id)
            im_h = self.load_image(self.HR_files(_))
            im_l = self.load_image(self.LR_files(_))
        
        return im_h, im_l

    def match_altered(self, update=True, paths=False):
        # Get the files that have been generated in the output directory
        # If update is false, then just return a list of matched names, if true then
        # change the class variable values accordingly.
        hr_files = self._get_HR_out_()
        lr_files = self._get_LR_out_()

        # Get a set of all the files with agreement before the metadata
        if len(hr_files) > len(lr_files):
            matches = list(set(hr_files)-(set(hr_files)-set(lr_files)))
        else:
            matches = list(set(lr_files)-(set(lr_files)-set(hr_files)))

        if update:
            # If you want to save these matched files as class variables
            self.HR_files = [self.HR_out_dir + _ for _ in matches]
            self.LR_files = [self.LR_out_dir + _ for _ in matches]
            print('HR and LR file locations updated')
        
        if paths:
            return self.HR_files, self.LR_files

    def change_out(self, HR_out_dir, LR_out_dir):
        # Change the output locations so you can save into a new file
        self.HR_out_dir = HR_out_dir
        self.HR_files = None
        self.LR_out_dir = LR_out_dir
        self.LR_files = None

### ACTUAL IMAGE MANIPULATION BELOW

    def run(self, clear=False, save=False):
        # This method is called to generate the data

        if clear:
            print('Clearing existing output directories')
            shutil.rmtree(self.HR_out_dir, ignore_errors=True)
            if self.template['resolution'] != None:
                shutil.rmtree(self.LR_out_dir, ignore_errors=True)
                os.makedirs(self.LR_out_dir, exist_ok=True)

        os.makedirs(self.HR_out_dir, exist_ok=True)
        fnames_h = []
        fnames_l = []

        for ids, im in enumerate(self.inp_paths):
            im_h = self.load_image(im)
            opp, im_h = self.img_transform(im_h)

            # Prevents weird new file naming issues when input is compressed file (.nii.gz)
            im = Path(im)
            while im.suffix in {'.tar', '.gz', '.zip'}:
                im = im.with_suffix('')
            
            im = os.path.splitext(im)[0]+opp # Add transformations to file name
            im = os.path.split(im)[1]
        
            # Generate Low Resolution
            if self.template['resolution']:

                dim = im_h.shape
                # efficient way to either make a single value into an array or do nothing if resolution is already a vector
                # TODO: replace transformation if statements with this
                self.template['resolution'] = [int(x) for x in np.multiply(np.ones(len(dim)), self.template['resolution'])]

                # Check that dimensions of HR image are multiples of resolution change, else shave off data
                print(dim)
                print(self.template['resolution'])
                for i in range(len(dim)):
                    if dim[i] % self.template['resolution'][i]:
                        # If it isn't a clean scaling down
                        _ = dim[i]-(dim[i] % self.template['resolution'][i])

                        im_h = np.delete(im_h,[x for x in range(_, dim[i])],i)

                im_l = self.gen_LR_img(im_h, self.template['resolution'])

            # Create image patches and save them
            if self.template['patch'] and save:
                fnames_h, slice_select = self.img2patches(im_h, self.HR_out_dir + im, save=True, sanity_check=True)
                print(f'patches to keep = {slice_select[:25]}')
                if self.template['resolution'] != None:
                    fnames_l = self.img2patches(im_l, self.LR_out_dir + im, same_size=self.template['same_size'], save=True, slice_select=slice_select, sanity_check=False)

                    # if not _a == _:
                    #     raise FileExistsError('''WARNING: The patches for High and Low resolution do not match, this is
                    #         most likely due to resolution scaling or patches/steps not being divisible by resolution''')

            elif save:
                fname_h = self.HR_out_dir + im
                self.save_image(fname_h, im_h)
                fnames_h.append(fname_h)
                if self.template['resolution']:# != None:
                    fname_l = self.LR_out_dir + im
                    self.save_image(fname_l, im_l)
                    fnames_l.append(fname_l)


        self.HR_files = fnames_h
        self.LR_files = fnames_l

        print('Files processed successfully')



    def load_image(self, im_path):
        # Given an image path, determines the function required to load the contents
        # as a numpy array, which is returned.
        fil_typ = os.path.splitext(im_path)[1]

        if fil_typ == '.png':
            # If file is a png
            print(f'Loading {im_path} as png')
            img = np.array(Image.open(im_path))
            print(f'Image shape:{img.shape}')

            if self.template['unit'] == 'intensity':
                img = self.rgb2ycrbcr(img)
                img = img[:,:,0] #Just deal with intensity values at the moment because 
                                 # having multiple channels throws off cv2 when saving, 
                                 # since it also does BGR instead of RGB and will save a blue image
            elif self.template['unit'] == 'color':
                pass

        elif fil_typ == '.nii' or fil_typ == '.gz':
            print(f'Loading {im_path} as nii')
            img = nib.load(im_path).get_fdata()

        elif fil_typ == '.dcm':
            img = pydicom.dcmread(im_path).pixel_array
            print(f'Loading {im_path} as dicom')

        else:
            raise FileNotFoundError(f'Image file type {fil_typ} not supported.')

        return img

    def gen_LR_img(self, im_h, res, interp=1):
        # Generate the low-resolution image from the corresponding HR image using resizing
        dim = im_h.shape

        # TODO: Patching error occurs when the shape of an image has dimension of odd magintude with
        #       even 'res', and vice versa. Need to come up with a fix for this...
        new_dims = [math.floor(x) for x in np.divide(dim, res)]

        im_l = resize(im_h, new_dims, order = interp, mode='symmetric')

        if self.template['same_size']:
            im_l = resize(im_l, dim, order= interp, mode = 'symmetric')

        return im_l

    def img_transform(self, im_h, save=False):
        # Transform the original files using a variety of methods
        # Have im_h and im_l be numpy arrays
        opp = '' #string for storing the operations performed on the images

        dim = im_h.shape
        if len(dim)>3:
            raise ValueError('Dimension of input data not currently supported')
        
        # If single image is provided for any of these settings, convert into list of N dimensions
        if self.template['translation'] == None:
            trans = [None]
        elif type(self.template['translation']) != list:
            trans = [self.template['translation'] for _ in range(len(dim))]
        else:
            trans = self.template['translation'][:] #Weird thing I have to add to not link changes to 'trans' to self.template
        
        for idx, x in enumerate(trans):
            if trans[idx] != 0 and trans[idx] != None:
                trans[idx] = np.random.randint(-x,x)


        # Rotation
        if self.template['rotation'] == None:
            rot = [None]
        elif type(self.template['rotation']) != list:
            rot = [self.template['rotation'] for _ in range(len(dim))]
        else:
            rot = self.template['rotation'][:]

       
        for idx, x in enumerate(rot):
            if x != 0 and x != None:
                rot[idx] = np.random.randint(-x,x)


        # Scaling
        if self.template['scale'] == None:
            scale = [None]
        elif type(self.template['scale']) != list:
            scale = [self.template['scale'] for _ in range(len(dim))]
        else:
            scale = self.template['scale'][:]

        for idx, x in enumerate(scale):
            if scale[idx] != None and scale[idx] > 1:
                scale[idx] = np.random.randint(1,x+1)
            else:
                scale[idx] = 1



        # TODO: Issue with low resolution not necessairly having the same dimensions
        # Rotation 2D
        if len(rot) == 2 and type(rot[0])==int: #Make sure rotation isn't None for some reason
            im_h = rotate(im_h, rot[0], order=1)
            opp += f'_rot{rot[0]}'
        # Rotation 3D
        elif len(rot) == 3:
            for i in range(dim[0]):
                im_h[i,:,:] = rotate(im_h[i,:,:],rot[0], order=1)
            for i in range(dim[1]):
                im_h[:,i,:] = rotate(im_h[:,i,:],rot[1], order=1)
            for i in range(dim[2]):
                im_h[:,:,i] = rotate(im_h[:,:,i],rot[2], order=1)
            opp += f'_rot{rot[0]}_{rot[1]}_{rot[2]}'
            

        # Translation
        if len(trans) == 2:
            transform = AffineTransform(translation=(trans[0], trans[1]))
            im_h = warp(im_h, transform, mode="symmetric")
            opp += f'_tr{trans[0]}_{trans[1]}'
        elif len(trans) == 3:
            transform = AffineTransform(translation=(trans[1], trans[2]))
            for i in range(dim[0]):
                im_h[i,:,:] = warp(im_h[i,:,:],transform, mode = 'symmetric')

            for i in range(dim[1]):
                # Because two dimensions were already translated, you only need to translate
                # along one dimension
                im_h[:,i,:] = warp(im_h[:,i,:], AffineTransform(translation=(trans[0],0)), mode='symmetric')
                
            opp += f'_tr{trans[0]}_{trans[1]}_{trans[2]}'

        # Scaling
        if len(scale) >= 2:
            #transform = AffineTransform(scale=_a)
            im_h = rescale(im_h, scale = scale, mode='symmetric')
            
            # Dumb way to make added string fit based on scaling
            try: opp+= f'_scale_{scale[0]}_{scale[1]}_{scale[2]}'
            except: opp+= f'_scale_{scale[0]}_{scale[1]}'

        return opp, im_h

    def img2patches(self, im_h, fname, same_size=True, keep_blank=False, slice_select=None, save=False, sanity_check=False):
        # Depending on the number of dimenions in the `patch` value, either make 2D
        # or 3D images

        dim = im_h.shape
        print(f'shape of image = {dim}')
        patch_size = self.template['patch'][:]
        step = self.template['step'][:]

        
        im_name=Path(fname).with_suffix('').__str__()
        #im_name = fname.split('.')[:-2][0] #Kind of janky way to just strip away the suffix
        
        if slice_select: #If slice_select is provided, then getting rid of blanks really screws things up
            print('keeping blank')
            keep_blank = True

        if type(patch_size) != list:
            patch_size = [patch_size for _ in range(len(dim))]
        
        if type(step) != list:
            step = [step for _ in range(len(dim))]

        # Whether to shrink the patch size and step size down by the scaling amount for LR images without the same dimensions as the HR images
        if not same_size:
            try: 
                patch_size = [math.floor(x) for x in np.divide(patch_size,self.template['resolution'])]
                print(f'patch size = {patch_size}')
            except: raise ValueError(f'Resolution change coefficient: {self.template["resolution"]} not defined properly for patch_size: {patch_size}')

            try: 
                step = [math.floor(x) for x in np.divide(step,self.template['resolution'])]
                print(f'step size = {step}')
            except: raise ValueError(f'Resolution change coefficient: {self.template["resolution"]} not defined properly for step: {step}')
        else:
            print(f'patch size = {patch_size}')
            print(f'step size = {step}')

        # Create a numpy stack following Pytorch protocols, so 1 dimension more than patch
        
        # Count number of non-zero entries
        cnt = 0
        blank = 0
        not_blank = []
        itter = -1

        # Get total number of patches that will be created:
        patch_count = np.prod([len(range(0,i,step[idx])) for idx, i in enumerate(dim)])
        print(f'patch guess = {np.prod([math.floor((i-patch_size[idx])/step[idx])+1 for idx,i in enumerate(dim)])}')
        patch_vol = math.prod(patch_size)*0.4 #TODO: make value changable

        if len(dim) == 2:
            stack = np.zeros(patch_count,patch_size[0],patch_size[1])
            print(f'stack size = {stack.shape}')

            for i in range(0,dim[0],step[0]):
                for j in range(0,dim[1],step[1]):
                    if i+patch_size[0] <= dim[0] and j+patch_size[1] <= dim[1]:
                        itter = itter+1 #just a calculator for finding when blanks occur
                        samp = im_h[i:i+patch_size[0],j:j+patch_size[1]]

                        if keep_blank or (samp==0).sum() <= patch_vol:#(samp.max() > 0):
                            stack[cnt,:,:] = samp
                            cnt += 1
                            not_blank.append(itter)
                        else:
                            blank += 1
                            #blank.append(_)
        elif len(dim) == 3:
            stack = np.zeros((patch_count,patch_size[0],patch_size[1], patch_size[2]))
            print(f'stack size = {stack.shape}')

            for i in range(0,dim[0],step[0]):
                for j in range(0,dim[1],step[1]):
                    for k in range(0,dim[2],step[2]):
                        #itter = itter+1 #just a calculator for finding when blanks occur
                        if i+patch_size[0] <= dim[0] and j+patch_size[1] <= dim[1] and k+patch_size[2] <= dim[2]:
                            itter = itter+1
                            samp = im_h[i:i+patch_size[0],j:j+patch_size[1], k:k+patch_size[2]]

                            if keep_blank or (samp==0).sum() <= patch_vol:#(samp.max() > 0):
                                stack[cnt,:,:,:] = samp
                                cnt += 1
                                not_blank.append(itter)
                            else:
                                blank += 1
                                #blank.append(_)
            print(itter)
        else:
            raise IndexError(f'Images of dimension {dim} not supported by this method. Only 2D and 3D data accepted.')
        

        #TODO: There MUST be a better way to organize this whole mess, lol

        fnames = []
        if slice_select:
            for i in range(len(slice_select)):
                fnames.append(f'{im_name}_{i}.{self.template["out_type"]}')
        else:
            for i in range(cnt):
                fnames.append(f'{im_name}_{i}.{self.template["out_type"]}')

        if save:
            if slice_select:
                for idx, i in tqdm(enumerate(slice_select)):
                    self.save_image(fnames[idx], stack[i], verbose=False)
            else:
                for idx, i in tqdm(enumerate(fnames)):
                    self.save_image(i,stack[idx], verbose=False)
            if sanity_check:
                print(f'Number of patches: {len(not_blank)}')
                print(f'Number of blank patches: {blank}')
                return fnames, not_blank
            else:
                return fnames
        else:
            if sanity_check:
                return fnames, stack, not_blank
            else:
                return fnames, stack

    def rgb2ycrbcr(self, img_rgb):
        #Takes an RBG image and returns it as a YCRBCR image 
        # (if you just want to focus on luminance values of an image)

        img_rgb = img_rgb.astype(np.float32)
        
        img_ycrcb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YCR_CB)
        img_ycbcr = img_ycrcb[:,:,(0,2,1)].astype(np.float32)
        img_ycbcr[:,:,0] = (img_ycbcr[:,:,0]*(235-16)+16)/255.0
        img_ycbcr[:,:,1:] = (img_ycbcr[:,:,1:]*(240-16)+16)/255.0

        return img_ycbcr

    def load_image_pair(self, im_id):
        # A method which loads the provided file and returns a numpy array
        # this is used because it will remember in the template dictionary how
        # the images were saved (either as RBG or intensity values or 3D array). 
        # This will help minimize headaches caused by different image types.

        # im_id can either be the index value or the name of the file
        if isinstance(im_id, int):
            HR_file = self.HR_files[im_id]
            LR_file = self.LR_files[im_id]
        elif isinstance(im_id, str):
             _ = self.HR_files.index(im_id)
             HR_file = self.HR_files[_]
             LR_file = self.LR_files[_]
        else:
            TypeError("Invalid image identifier, please input a string to integer")

        im_h = self.load_image(HR_file)
        im_l = self.load_image(LR_file)

        return im_h, im_l


    def save_image(self, fname, im, verbose = False):
        # Take a given image and save it as the specified format:
        # fname = output name of the saved file
        # im = numpy array of image

        form = self.template['out_type']

        dim = im.shape #Get number of dimensions of image

        if form == 'png':
            # Check that you aren't saving a 3D image
            #TODO: Scale inputs to [0,255] so data isn't lost/image isn't saturated
            cv2.imwrite(f'{fname}',im)
            print(f'Saving: {fname}')
        elif form == 'nii':

            # TODO: Add option to transpose image for some reason because mricron hates the first dim[0] = 1
            # Still gets loaded fine in terms of loading into python, but visualizing it is bad
            # np.transpose(im, (1,2,0))


            # TODO: If image is 2D then append a third  dimension before saving(?)
            nib.save(nib.Nifti1Image(im, np.eye(len(dim)+1)), fname)
            if verbose:
                print(f'Saving: {fname}')
        elif form == 'dcm':
            raise NotImplementedError('DICOM saving currently not supported')
        else:
            raise NotImplementedError('Specified file type is currently not supported for saving')


testclass = SrGen('../data/CNNIL_nifti/Raw/','../data/CNNIL_nifti/HR_patches_ax/','../data/CNNIL_nifti/LR_patches_ax/')
temp = testclass.get_template()
temp['out_type'] = 'nii'
temp['resolution'] = [2,2,1]
temp['translation'] = [10, 10, 10]
temp['rotation'] = [0, 0, 0]
# temp['scale']= [1,1,1]
temp['keep_blank'] = False
temp['same_size'] = False
temp['patch'] = [18,18,1] #[x,y,z] when looking at the brain from the top down
temp['step'] = [10,10,2]
testclass.set_template(temp)

testclass.run(clear=True, save=True)



Clearing existing output directories
Loading ../data/CNNIL_nifti/Raw/29006_anat.nii.gz as nii
(193, 256, 256)
[2, 2, 1]
shape of image = (192, 256, 256)
patch size = [18, 18, 1]
step size = [10, 10, 2]
patch guess = 55296
stack size = (66560, 18, 18, 1)
55295


14660it [00:11, 1282.32it/s]


Number of patches: 14660
Number of blank patches: 40636
patches to keep = [6727, 6835, 6836, 6838, 6839, 6840, 6841, 6842, 6843, 6844, 6845, 6846, 6847, 6848, 6849, 6850, 6851, 6852, 6853, 6854, 6855, 6856, 6857, 6858, 6859]
shape of image = (96, 128, 256)
keeping blank
patch size = [9, 9, 1]
step size = [5, 5, 2]
patch guess = 55296
stack size = (66560, 9, 9, 1)
55295


14660it [00:11, 1332.56it/s]

Files processed successfully


In [26]:
q = np.array([2,3,4])
[_ for _ in range(int(q[0]-1),int(q[2]))]

[1, 2, 3]

In [32]:
list(range(0,4))

[0, 1, 2, 3]

In [42]:
testclass.match_altered(update=False, paths=True)

2

In [84]:
np.transpose(testclass.load_image('../data/CNNIL_nifti/HR_patches_ax/29006_anat_rot0_0_0_tr1_0_8_scale_1_1_1_11.nii'),(2,0,1))
# testclass.load_image('../data/CNNIL_nifti/HR_patches_ax/29006_anat_rot0_0_0_tr1_0_8_scale_1_1_1_11.nii')

Loading ../data/CNNIL_nifti/HR_patches_ax/29006_anat_rot0_0_0_tr1_0_8_scale_1_1_1_11.nii as nii


memmap([[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]])

In [69]:
for i in range(0,1,1):
    print(i)
# if not a == b:
#     raise FileExistsError('''WARNING: The patches for High and Low resolution do not match,
#         most likely due to resolution scaling or patches/steps not being divisible by resolution''')

0


In [31]:

a = np.zeros((2,2))
a[1,0]=2
a[0,0]=3
resize(a,(4,4), order=1, mode='symmetric')


array([[3.    , 2.25  , 0.75  , 0.    ],
       [2.75  , 2.0625, 0.6875, 0.    ],
       [2.25  , 1.6875, 0.5625, 0.    ],
       [2.    , 1.5   , 0.5   , 0.    ]])

In [34]:
if True or (a.max() >0):
    print('this')

this


In [29]:
filename = Path('/this/right/here')
filename.with_suffix('').__str__()

'/this/right/here'

## Test with Dataloader and NN model
Here we'll make sure the above class works in a way that is compatible with Pytorch's Dataloader and ML modeling

In [ ]:
# TODO: There's not reasing I can't combine the Dataset class and my custom class into one thing

class Dataset(torch.utils.data.Dataset):
    def __init__(self, sr_class):
        self.sr_class = sr_class

        # In case I forget to run match_altered before pulling the class
        if not sr_class.HR_files:
            sr_class.match_altered(update=True)

    def __len__(self):
        return len(self.sr_class.HR_files)

    def __getitem__(self, index):
        Y, X = self.sr_class.load_image_pair(index)
        X = torch.unsqueeze(torch.tensor(X, dtype=torch.float32),0)
        Y = torch.unsqueeze(torch.tensor(Y, dtype=torch.float32),0)

        return X, Y

In [ ]:
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 0}

training_set = Dataset(qq)
training_generator = torch.utils.data.DataLoader(training_set, **params)

In [ ]:
class SCN(nn.Module):
    def __init__(self,sy,sg, model_file=False, train=True):
        super().__init__()
        C = 5
        L = 5

        Dx = torch.normal(0,1, size = (25,128))
        Dy = torch.normal(0,1, size = (100,128))
        I = torch.eye(128)

        self.conv = nn.Conv2d(1,100,9, bias = False, stride =1, padding = 6)
        self.mean2 = nn.Conv2d(1,1,13, bias = False, stride = 1, padding = 6)
        self.diffms = nn.Conv2d(1,25,9, bias=False, stride = 1, padding=6)

        self.wd = nn.Conv2d(100,128,1,bias = False, stride = 1)
        self.usd1 = nn.Conv2d(128, 128, 1, bias = False, stride=1)
        self.ud = nn.Conv2d(128,25,1,bias=False,stride=1)
        self.addp = nn.Conv2d(16,1,1, bias = False, stride = 1)

        if train: #If you are currently training the model
            self.mean2.weight = torch.nn.Parameter(self.create_gaus(13), requires_grad = False)
            self.diffms.weight = torch.nn.Parameter(self.create_diffms(9,5),requires_grad=False)
            self.wd.weight = torch.nn.Parameter(self.expand_params(C*Dy.T), requires_grad=True)
            self.usd1.weight = torch.nn.Parameter(self.expand_params(I - torch.matmul(Dy.T,Dy)), requires_grad=True)
            self.ud.weight = torch.nn.Parameter(self.expand_params((1/(C*L))*Dx), requires_grad=True)
            self.addp.weight = torch.nn.Parameter(torch.ones(1,16,1,1)*0.06, requires_grad=True)

        else:
            self.conv.weight = torch.nn.Parameter(torch.ones(100,1,9,9),requires_grad=False)
            self.mean2.weight = torch.nn.Parameter(self.create_gaus(13),requires_grad=False)
            self.diffms.weight = torch.nn.Parameter(self.create_diffms(9,5),requires_grad=False)
            self.wd.weight = torch.nn.Parameter(self.expand_params(C*Dy.T),requires_grad=False)
            self.usd1.weight = torch.nn.Parameter(self.expand_params(I - torch.matmul(Dy.T,Dy)),requires_grad=False)
            self.ud.weight = torch.nn.Parameter(self.expand_params((1/(C*L))*Dx),requires_grad=False)
            self.addp.weight = torch.nn.Parameter(torch.ones(1,16,1,1)*0.06,requires_grad=False)


    def forward(self, x, k, sy=9, sg=5):
        im_mean = self.mean2(x)
        # print(f'im_mean shape {im_mean.shape}')
        diffms = self.diffms(x)
        # print(f'diffms shape: {diffms.shape}')

        n, c, h, w = x.shape
        # y = torch.zeros(n, 100, h-8, w-8)
        x = self.conv(x)
        # print(f'post conv shape {x.shape}')
        #print(f'conv max {x.max()}')
        x=x+1

        x = x/torch.linalg.vector_norm(x, ord=2, dim=1, keepdim=True)
        # print(f'post vector norm shape: {x.shape}')
        #print(f'postnorm max {x.max()}')

        x = self.wd(x)
        #print(f'conv wd {x.max()}')
        z = self.ShLU(x,1)
        #print(f'conv SHLU {x.max()}')

        # Go through LISTA
        for i in range(k):
            z = self.ShLU(self.usd1(z)+x,1)

        x = self.ud(z)
        #print(f'ud max {x.max()}')
        # print(f'post ud shape {x.shape}')
        x = (x/torch.linalg.vector_norm(x, ord=2, dim=1, keepdim=True))*torch.linalg.vector_norm(diffms, ord=2, dim=1, keepdim=True)*1.1
        # print(f'prereassembled x shape {x.shape}')
        x = self.reassemble2(x,im_mean,4)
        # print(f'reassembled x shape {x.shape}')
        x = self.addp(x)
        #print(f'x.reassemble.max = {x.max()}')
        x = x+im_mean

        return x

    def reassemble2(self, x, im_mean, patch_size):
        img = im_mean
        s, c, h, w = img.shape
        
        # img_stack=torch.zeros(s,25,h,w)
        img_stack=torch.zeros(s,16,h,w)
        
        #go through every sample and reassemble the image
        for q in range(x.shape[0]):
            filt = 0
            for ii in range(patch_size-1, -1, -1):
                for jj in range(patch_size-1, -1, -1):
                    img_stack[q,filt,:,:] = x[q,filt,jj:(jj+h), ii:(ii+w)]
                    filt+=1
        
        return img_stack
    
    def create_diffms(self, kern_size, sy=5):
        diffms = torch.zeros(sy**2,1,kern_size,kern_size)
        
        neg = -1*(1/(sy**2))
        pos = 1+neg
        
        border = int((kern_size-sy)/2)
        base = torch.zeros(sy,sy)+neg
        cnt=0
        
        for i in range(sy**2):
            base = torch.zeros(sy**2)+neg
            base[cnt]=pos
            diffms[i,0,border:(kern_size-border),border:(kern_size-border)] = base.reshape([sy,sy])
            cnt+=1
        return diffms
    
    
    def create_gaus(self, kern_size, sy=9,std=2.15):
        n = torch.arange(0,sy)-(sy-1.0)/2.0
        sig2 = 2 * std * std
        gkern1d = torch.exp(-n ** 2 / sig2)
        gkern1d = gkern1d/torch.sum(gkern1d)
        #print(gkern1d.shape)
        gkern2d = torch.outer(gkern1d, gkern1d)
    

        # Wrap in zeros, if kern_size > sy
        gaussian_filter = torch.zeros(1,1,kern_size,kern_size)
        border = int((kern_size-sy)/2)
        gaussian_filter[0,0,border:(kern_size-border),border:(kern_size-border)] = gkern2d#(sy,std=std)
        #print(gaussian_filter.shape)
        return gaussian_filter
        
    
    def fixed_positions(self, tens, mult, sg):
        f, _ , h, w = tens.shape
        new_filt = torch.zeros(f*mult, 1, sg,sg)
        cnt = 0
        filt = 0
        
        for filt in range(f):
            for j in range((sg-w)+1):
                for i in range((sg-h)+1):
                    new_filt[cnt,0,i:i+h,j:j+w] = tens[filt]
                    cnt+=1
        return new_filt
    
    def expand_params(self,tens):
        return torch.unsqueeze(torch.unsqueeze(tens,2),3)
    
    def ShLU(self,a, th):
        return torch.sign(a)*torch.maximum(abs(a)-th, torch.tensor(0))

## Set Optimization Parameters

In [ ]:
net = SCN(9,5,train=True)
criterion = nn.MSELoss()

optimizer = optim.SGD(
    [
        {"params": net.addp.parameters()},#, "lr": 0.0002, "momentum": 0.00005},
        {"params": net.conv.parameters()},#, "lr": 0.0003, "momentum": 0.0001},
        {"params": net.wd.parameters()},
        {"params": net.usd1.parameters()},
        {"params": net.ud.parameters()},
    ],
    lr=0.00007, momentum = 0.0001
)

## Training Loop

In [ ]:
from tqdm import tqdm
import time
# Loop over epochs

max_epochs = 20

for epoch in tqdm(range(max_epochs)):
    losses = []
    losses_per = []

    # Training
    count = 0
    for inp, goal in training_generator:
        optimizer.zero_grad()

        output = net(inp,2) # the 2 is the number of iterations in the LISTA network
        print(output.shape)
        output = torch.clamp(output, 0, 255)

        loss = criterion(output,goal)
        loss.backward()
        optimizer.step()
        print(f'loss = {loss.item()}')
        losses.append(loss.item())
        print(f'mini-batch # {count}, mean loss = {sum(losses)/len(losses)}')
        count = count+1

    torch.save(net.state_dict(), f'./MRI_save_{epoch}.p')
    print(f'\n\n epoch {epoch}, loss mean: {sum(losses)/len(losses)}, loss: {min(losses)}-{max(losses)}\n')

    # Give computer time to cool down
    time.sleep(10)

## Testing Loop